In [23]:
# testset은 labedled로 가져감
def create_self_supervised_set(base_dir, target_data_name, self_train_num_per_class, self_test_num_per_class):
    ## 하나의 파일로 합치는 작업
#     dataroot = '../../../../dataset'
    dataroot = os.path.join(base_dir, 'train_size_500', 'train',)
    target_names = os.listdir(dataroot)
    print('>> check class : ', os.listdir(dataroot))
    
    file_train_names_list = list()
    file_test_names_list_ray = list()
    file_test_names_list_stomic = list()

    
    for target_name in target_names : 
        
        target_path = os.path.join(dataroot, target_name)
        
        target_train_filenames = sorted(os.listdir(target_path))[:self_train_num_per_class]
        target_test_filenames = sorted(os.listdir(target_path))[self_train_num_per_class:self_train_num_per_class+self_test_num_per_class]
        
        for target_filename in target_train_filenames : 
            target_filepath = os.path.join(target_path, target_filename)
            file_train_names_list.append(target_filepath)
        print(target_test_filenames)
        for test_filename in target_test_filenames : 
            if target_name == 'ray' :
                target_filepath = os.path.join(target_path, target_filename)
                file_test_names_list_ray.append(target_filepath)
            elif target_name == 'stonic' :
                target_filepath = os.path.join(target_path, target_filename)
                file_test_names_list_stomic.append(target_filepath)

    # 저장할 파일 생성
    save_base = os.path.join(base_dir, target_data_name, 'train_size_' + str(self_train_num_per_class))
    train_save_dir_out = os.path.join(save_base, 'train/')
    train_save_dir = os.path.join(save_base, 'train/images/')
    
    if not os.path.exists(train_save_dir) :
        os.makedirs(train_save_dir)
    # train 파일 저장
    for src_filename in file_train_names_list:
        img = cv2.imread(src_filename)
        cv2.imwrite(os.path.join(train_save_dir, src_filename.split('/')[-1]), img)

    
    # 저장할 파일 생성
    test_save_dir_out = os.path.join(save_base, 'test/')
    test_save_dir_ray = os.path.join(save_base, 'test/ray/')
    test_save_dir_stonic = os.path.join(save_base, 'test/stonic')
    
    if not os.path.exists(test_save_dir_ray) :
        os.makedirs(test_save_dir_ray)
    if not os.path.exists(test_save_dir_stonic) :
        os.makedirs(test_save_dir_stonic)
        
    # test 파일 저장
    print(file_test_names_list_ray)
    for src_filename in file_test_names_list_ray:
        img = cv2.imread(src_filename)
        cv2.imwrite(os.path.join(test_save_dir_ray, src_filename.split('/')[-1]), img)
        
    for src_filename in file_test_names_list_stomic:
        img = cv2.imread(src_filename)
        cv2.imwrite(os.path.join(test_save_dir_stonic, src_filename.split('/')[-1]), img)
    
    return train_save_dir_out, test_save_dir_out

In [24]:
def test(dataloader, net, device):

    net.eval()
    test_losses = list()
    test_trues = list()
    test_preds = list()

    for idx, (img, label) in enumerate(dataloader):

        img = img.to(device)
        label = label.to(device)

        out = net(img)
        
        softmax_out = torch.softmax(out, 1)
        softmax_max = softmax_out.detach().cpu().numpy().max()

        _, pred = torch.max(out, 1) # make prediction

        criterion = torch.nn.CrossEntropyLoss()
        
        loss = criterion(out, label)

        test_losses.append(loss.item())
        test_trues.extend(label.view(-1).cpu().numpy().tolist())
        test_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())


    acc, f1, auc, prec, rec = calculate_metrics(test_trues, test_preds)
    print(confusion_matrix(test_trues, test_preds))

    return net, acc, prec, rec, f1

In [25]:
def calculate_metrics(trues, preds):
    accuracy = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds, average='macro')
    precision = precision_score(trues, preds, average='macro')
    recall = recall_score(trues, preds, average='macro')

    return accuracy, f1, auc, precision, recall

In [26]:
import torch
import sys
import yaml
from torchvision import transforms, datasets
import torchvision
import numpy as np
import cv2
import os
import torchvision.models as models
from sklearn import preprocessing
from torch.utils.data.dataloader import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from PIL import Image

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import pickle

import os
import shutil
import numpy as np
from statistics import mean
import copy

from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import *

device = "cuda:0"

## testset : 클래스별 폴더로 다시 나눔
batch_size = 512
data_transforms = torchvision.transforms.Compose([transforms.Resize((256, 256)), 
                                                  transforms.ToTensor()])

# dataset_path = '../datasets/carclass_binary/ray_stonic/train_size_500/test'
dataset_path = './dataset/temp_ray_v_stonic/train_size_500/test'

train_dir, test_dir = create_self_supervised_set(base_dir = './dataset/temp_ray_v_stonic', target_data_name = 'temp_ray_v_stonic' , self_train_num_per_class = 500, self_test_num_per_class = 100)

test_dataset = datasets.ImageFolder(root=test_dir, transform = data_transforms)
test_loader = DataLoader(test_dataset, batch_size=batch_size,num_workers=0, drop_last=False, shuffle=True)

## Resnet 불러와서 model param
net = models.resnet50(pretrained=True)
best_checkpoint = '../byol_model.pt'
net.load_state_dict(torch.load(best_checkpoint, map_location=device))
net.to(device)

## 결과 확인
net, test_acc, test_prec, test_rec, test_f1 = test(test_loader, net, device)

>> check class :  ['images']
['20180617_11474_13898837_d1d72a2d3b85eb550ff80c93c5e4be6f.jpg', '20180617_11474_13898839_edc983ccd9ea690954629dfb5535d1a8.jpg', '20180617_11474_13898840_0932b489386a5f6239e11cf7531b75d3.jpg', '20180617_11474_13898841_494d1b820833ea9324d4d225ff5af519.jpg', '20180617_11497_13896910_34ab7ca3a4896c32c34234e64b1f6d2c.jpg', '20180617_11497_13896911_901127ca8615d3c07924fbdf06aad11f.jpg', '20180617_11497_13896912_7d27f1e3651eb9b90f0653174ba323d1.jpg', '20180617_11497_13896913_90076cefc41dce749561e4eb6289c24b.jpg', '20180617_11497_13896914_6d57ce48eef7ce9897bb5a43ef0bb4f3.jpg', '20180617_11497_13896915_d64bc9086594dbcb4e740eae26f024f1.jpg', '20180617_11500_13897203_73f3f32391c1e9adc098f42ac6db26e0.jpeg', '20180617_11500_13897204_406b18aaf3d2a39c3f616de75a24ae10.jpeg', '20180617_11500_13897205_05b98d4f49bda36e479132354bf2ded2.jpeg', '20180617_11500_13897206_a7485514da14cb61783abb6d2d32942a.jpeg', '20180703_1655_14267133_f8837a8a2b533a6faa7ac54e3282a55b.jpg', '201807

RuntimeError: Found 0 files in subfolders of: ./dataset/temp_ray_v_stonic/temp_ray_v_stonic/train_size_500/test/
Supported extensions are: .jpg,.jpeg,.png,.ppm,.bmp,.pgm,.tif,.tiff,.webp